# Оцінка моделей

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

__Функція для виведення класифікаційного звіту__

In [ ]:
def model_classification_report(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    print(
        f"Класифікаційний звіт для моделі {model_name}:\n",
        classification_report(y_test, y_pred),
    )